In [ ]:
!git clone https://github.com/leonardozilli/FrameTrigger.git
%cd FrameTrigger/

In [1]:
!pip install transformers[torch] datasets -q

In [ ]:
from src.fn17 import load_dataset_hf, load_dataset_nltk
from src.train import train, test
from src.predict import predict
from src.process_data import prepare_data

In [ ]:
dataset = load_dataset_nltk(load_exemplars=False)

In [30]:
CHECKPOINT = 'bert-base-cased' # 'bert-base-cased', 'roberta-base', 'distilbert-base-cased'
TASK = 'frames' # 'triggers' or 'frames'

In [ ]:
tokenized_dataset = prepare_data(dataset, CHECKPOINT, task=TASK)
tokenized_dataset

## Train

In [18]:
OUT_DIR = f'./models/{CHECKPOINT}/{TASK}'
N_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 2e-5

In [ ]:
train(pretrained_model=CHECKPOINT, task=TASK, dataset=tokenized_dataset,
    epochs=N_EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RATE, model_output_path=OUT_DIR)

## Test

In [ ]:
test(OUT_DIR, tokenized_dataset['test'], task=TASK)

## Predict

In [35]:
MODEL_TARGETS = f'./models/{CHECKPOINT}/targets'
MODEL_FRAMES = f'./models/{CHECKPOINT}/frames'

sentences = [
    "When the moon hits your eye, that's 'amore'.",
    "The hallway smelt of boiled cabbage and old rag mats.",
    "Hoover Dam played a major role in preventing Las Vegas from drying up",
    "The price of bananas rose 5%",
    "The center's director pledged a thorough review of safety precedures"
]

for sentence in sentences:
    predict(pretrained_model=MODEL_TARGETS, sentence=sentence, task='targets', visualize=True)
    predict(pretrained_model=MODEL_FRAMES, sentence=sentence, task='frames', visualize=True)
    print('=====================')